In [71]:
import torch
import time
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.monte_carlo import qExpectedImprovement, qNoisyExpectedImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
NOISE_SE = 0.5

In [57]:
# the function to be maximized
from botorch.test_functions import Hartmann
neg_hartmann6 = Hartmann(negate=True)

In [58]:
def generate_initial_data(n):
    # generate training data
    train_x = torch.rand(n, 6, device=device, dtype=dtype)
    exact_obj = neg_hartmann6(train_x).unsqueeze(-1)  # add output dimension
    train_obj = exact_obj + NOISE_SE * torch.randn_like(exact_obj)
    best_observed_value = train_x.max().item()
    return train_x, train_obj

In [100]:
# initialize training data and the model
train_X, train_Y = generate_initial_data(n=10)

gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)
UCB = UpperConfidenceBound(gp, beta = 0.1)
candidate, acq_value = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    )

In [101]:
bounds = torch.stack([torch.zeros(6), torch.ones(6)])
gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
best_observed_value = []

for iteration in range(1, 40):
    t0 = time.time()
    
    fit_gpytorch_model(mll)
    
    UCB = UpperConfidenceBound(gp, beta = 1)    
    candidate, acq_value = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    )
    new_x = candidate.detach()
    print(new_x)
    new_y = neg_hartmann6(new_x).unsqueeze(-1)
    new_y = new_y + NOISE_SE * torch.randn_like(new_y)
    train_X = torch.cat([train_X, new_x])
    train_Y = torch.cat([train_Y, new_y])
    best_value = neg_hartmann6(train_X).max()
    best_observed_value.append(best_value)
    
    # update GP model using dataset with new datapoint
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    
    t1 = time.time()
    
    print(
            f"\nBatch {iteration:>2}: best_value = "
            f"({best_value:>4.2f}), "
            f"time = {t1-t0:>4.2f}.", end=""
        )

tensor([[0.0457, 0.1829, 0.0935, 0.6694, 0.1073, 0.6989]])

Batch  1: best_value = (0.64), time = 0.22.tensor([[0.0201, 0.2234, 0.0421, 0.6396, 0.2546, 0.7374]])

Batch  2: best_value = (0.64), time = 0.44.tensor([[0.0917, 0.1983, 0.1672, 0.6350, 0.0344, 0.6170]])

Batch  3: best_value = (0.64), time = 0.25.tensor([[0.6006, 0.5928, 0.8311, 0.3431, 0.0000, 0.0094]])

Batch  4: best_value = (0.64), time = 0.22.tensor([[0.5767, 0.6221, 0.7909, 0.4261, 0.0000, 0.0084]])

Batch  5: best_value = (0.89), time = 0.21.tensor([[0.6572, 0.5380, 0.9214, 0.1754, 0.0000, 0.0078]])

Batch  6: best_value = (0.89), time = 0.18.tensor([[0.5609, 0.4458, 0.8834, 0.1684, 0.0000, 0.0490]])

Batch  7: best_value = (0.89), time = 0.40.tensor([[0.0348, 0.1480, 0.0867, 0.7037, 0.0427, 0.7130]])

Batch  8: best_value = (0.89), time = 0.21.tensor([[0.7355, 0.6900, 0.9060, 0.2905, 0.0299, 0.0000]])

Batch  9: best_value = (0.89), time = 0.19.tensor([[0.1170, 0.2672, 0.1835, 0.5742, 0.1571, 0.5899]])

Batch 10: bes